In [2]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
import functools
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
from numba import njit,jit
import numpy as np
from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
import threading
import asyncio
# import grequests
from concurrent.futures import ThreadPoolExecutor, as_completed


simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

Entrez.tool = "Zoonosis predictor"

Entrez.email = input("Enter an email address to use NCBI e-utils: ")


In [3]:
def resetkmerdict(permset)->OrderedDict:
        kmerdict = OrderedDict()
        for i in permset:
            kmerdict[i]=0
        return kmerdict

def assign_kmers_to_dict(st, permset, kmer):
    kmerdict=resetkmerdict(permset)
    # st = row[2] # tune for which column the sequence is in
    for j in range(len(st)-kmer+1):
        if not st[j:j+kmer] in permset: continue
        kmerdict[st[j:j+kmer]]+=1
    return kmerdict

def getTrainParams(mergedDf):
    kmer = 4
    print(mergedDf)
    s = product('acgt',repeat = kmer)
    permset = set(["".join(x) for x in list(s)])
    # print(permset)

    l = []
    
    for row in tqdm.tqdm(mergedDf.itertuples()):
        # print(row)
        l.append(assign_kmers_to_dict(row[2], permset, kmer))
        

    finalkmerdict=pd.DataFrame(l)
    # print(finalkmerdict)

    # print("finished")
    mergedDf.fillna(0, inplace=True)

    X = finalkmerdict
    Y = mergedDf['isZoonotic']

    print(X)
    print(Y)
    return train_test_split(X, Y, test_size=0.2, random_state=1)

In [11]:
def queryKmer(ID, isZoonotic_list, index, everything):
    try:
        QueryHandle = Entrez.efetch(db="nucleotide", id=ID, 
                                    rettype="gb", retmode="text")
    except HTTPError as Error:
        if Error.code == 400:  # Bad request
            raise ValueError(f"Accession number not found: {ID}")
        else:
            raise

    SeqRec = SeqIO.read(QueryHandle, "genbank")
    # print(str(SeqRec.seq))
    everything.append({'accession': ID, 'sequence': str(SeqRec.seq).lower(), 'isZoonotic': isZoonotic_list[index]})
    

In [14]:
def getSequences(mergedDf):
    accession_list = [] # maintain order
    isZoonotic_list = [] # maintain order
    accession_set = set()
    isZoonotic_set = set()


    for row in tqdm.tqdm(mergedDf.itertuples()):
        # row[13] = accession, row[15] = infects humans
        for single_acc in row[14].split("; "):
            if single_acc not in accession_set:
                accession_list.append(single_acc)
                isZoonotic_list.append(row[15])
                accession_set.add(single_acc)
                isZoonotic_set.add(row[15])


    # TODO: RUN MULTIPLE THREADS TO SPEED UP
    threads = []
    vals = []
    
    for index, ID in enumerate(tqdm.tqdm(accession_list[:100])): #only read the first 100 lol
        # multithread for speed up
        queryKmer(ID, isZoonotic_list, index, vals)
        # x = threading.Thread(target=queryKmer, args=(ID, isZoonotic_list, index, vals))
        # threads.append(x)
        # x.start()

    # for index, thread in enumerate(tqdm.tqdm(threads)):
    #     thread.join()
    
    print(vals)
        

    df = pd.DataFrame(vals)

    return df
    


In [16]:
mergedDf = pd.read_csv("FinalData_Cleaned.csv")
sequences = getSequences(mergedDf)
X_train, X_test, y_train, y_test = getTrainParams(sequences)

913it [00:00, 552662.66it/s]
100%|██████████| 100/100 [00:51<00:00,  1.93it/s]


[{'accession': 'NC_025403.1', 'sequence': 'accagagggaaaatataacaatgtcgttttatagcgatgtaaataatacttatgtaggcccgaaagtggcatcaatcgtagtcgaagtcgactgcatcgagttgctttcggggcttgtatgatcttattcctccactagtggggatatccaggggtcatacaagatgtcttctgtctttaaagcctttgaacagttcactcttgaacaggatcaatatgtcggaggcaataatcttgaattgccgccagagacacttcaaactacaatcaaggttttcattcttaatacccagaaccctcaaacaaggtatcaaatgctttgcttctgcttgagaattatagctagtaactctgcaagaactgctcaaaaatatggtgcaatgctcacagtccttagtctacctactgctatgatgcacaaccacatgaggattgcagatcgttcacccgattcaatcatagaacgaattgagattgaaggttttgagccaggatcatacaggcttcgcccaaatgcaagaacacccatgacacaaggagaaatcatagcaatggaaaatatggctgatgatctgcctgacacaattgtcaaccaaaccccatttgttaacactgcaacagaggatgagatttgtgacgagatggaaaaattcttgaatgcagtctatagtgtgctaatccaggtctgggtcatggtgtgcaaatgcatgacagcatttgatcaacctacaggatctgatgcaaggagactagccaagtatcaacaacaaggacgtatggatcaaaagtatgtccttcagggtgaggttaggaggctaatacagttatgtatcagagaaagccttaccatcagacaatttttagttaatgagatgcagactgctagaagacaaggccccattacaggaaagtactatgctatggttggcgatgttgcaagatatatagagaatgcag

100it [00:00, 237.97it/s]

    tctc  cttg  cctc  tacg  gagt  agtc  agcg  caac  aaaa  atta  ...  ataa  \
0     54    62    38     7    62    49    10    81   148   111  ...    86   
1     58    82    39    13    63    45     8    90   131    99  ...    88   
2     35    43    22    13    57    45     9    49   296   187  ...   153   
3     19     5    35    13    23    15    25    40    26    11  ...    10   
4     16    10    32    13    23    16    23    45    30     3  ...    12   
..   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
95     5     9     0     7     9     5     6     7    16    18  ...    20   
96     3     7     1     5     5    12     6     7    15    22  ...    13   
97     3     7     2     3     7     8     4     5    20     9  ...     4   
98     6     6     0     6     7     3     5     8    15     8  ...    12   
99     3     8     0     3     4     0     0     6     6     9  ...     5   

    tcac  tccc  ttac  agcc  ccgc  cgta  gacc  cgcc  tggt  
0     66    35  

In [22]:
best_gradBoost = pickle.load(open('curr_models/gradBoost.pkl', 'rb'))
best_xgBoost = pickle.load(open('curr_models/xgBoost.pkl', 'rb'))


kmer = 4
print(sequences)

s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

pred_arr = []
xg_pred = []
for ind, file in enumerate(os.listdir("./virome_contigs")):
    fasta_sequences = SeqIO.parse(open(f"./virome_contigs/{file}"),'fasta')

    fasta = [x for x in fasta_sequences][0]
    # print(fasta)
    
    name, sequence = fasta.id, str(fasta.seq)
    X_info = sequence.lower()

    oDict = assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

    kmer_df = pd.DataFrame()

    for i in oDict:
        kmer_df.at[0, i]=oDict[i]
    cols_when_model_builds = best_gradBoost.feature_names_in_
    kmer_df=kmer_df[cols_when_model_builds]
    
    pred_arr.append(best_gradBoost.predict(kmer_df))
    
    cols_when_model_builds = best_xgBoost.get_booster().feature_names
    kmer_df=kmer_df[cols_when_model_builds]

    xg_pred.append(best_xgBoost.predict(kmer_df))


    
pred_arr = np.asarray(pred_arr)
xg_pred = np.asarray(xg_pred)


    # print(best_gradBoost.predict(kmer_df), sequences.loc[ind]['isZoonotic'])
    # print(best_gradBoost.predict(kmer_df), sequences['isZoonotic'])
        # print(accuracy_score())

print(sequences['isZoonotic'][sequences['isZoonotic'] == 1].to_numpy())
print(len(sequences['isZoonotic'][sequences['isZoonotic'] == 1].to_numpy()))
print(xg_pred.sum())
print(pred_arr.sum())


    # with open(output_file) as out_file:
    #     for fasta in fasta_sequences:
    #         name, sequence = fasta.id, str(fasta.seq)
    #         new_sequence = some_function(sequence)
    #         write_fasta(out_file)

      accession                                           sequence  isZoonotic
0   NC_025403.1  accagagggaaaatataacaatgtcgttttatagcgatgtaaataa...           1
1   NC_025404.1  accagagggaaaattaagaaaggtcgttccaagacgacttaaaaga...           1
2   NC_028246.1  acggagaaaaacaaaaaaactatagtgattagataaataaggaaaa...           1
3   NC_002077.1  ttgcccactccctctctgcgcgctcgctcgctcggtggggcctgcg...           1
4   NC_006152.1  ctctcccccctgtcgcgttcgctcgctcgctggctcgtttgggggg...           1
..          ...                                                ...         ...
95  NC_004218.1  gtattaaatttttgtaagtcgttatggaattatttagtgacagtgg...           1
96  NC_004219.1  gtatttaaaattcatgtttttgcatcatggcgtgggttacgcaagc...           1
97  NC_004220.1  gtattaaaaattacaagaacctaacattgcaatggagatcttgaga...           1
98  NC_004221.1  gtatttaaaattatagaaagttctgaacctaggggtctttctgtct...           1
99  NC_004204.1  gtattaaaattcagcaattgtccaatttaggaaacattctgtttaa...           1

[100 rows x 3 columns]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [20]:
print(accuracy_score(sequences['isZoonotic'], pred_arr[:100]))
print(accuracy_score(sequences['isZoonotic'], xg_pred[:100]))

0.33
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
0.07


In [42]:
def getSingleSequence(accessionID):
    try:
        QueryHandle = Entrez.efetch(db="nucleotide", id=accessionID, 
                                    rettype="gb", retmode="text")
    except HTTPError as Error:
        if Error.code == 400:  # Bad request
            raise ValueError(f"Accession number not found: {accessionID}")
        else:
            raise

    SeqRec = SeqIO.read(QueryHandle, "genbank")
    print(str(SeqRec))

    oDict = assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

    kmer_df = pd.DataFrame()

    for i in oDict:
        kmer_df.at[0, i]=oDict[i]
    # print(best_gradBoost.predict_proba(kmer_df))

    cols_when_model_builds = best_gradBoost.feature_names_in_
    kmer_df=kmer_df[cols_when_model_builds]
    
    print([round(x, 2) for x in best_gradBoost.predict_proba(kmer_df).tolist()[0]])
    print(best_gradBoost.predict(kmer_df).tolist()[0])

    # ls = []
    # ls.append({'accession': accessionID, 'sequence': str(SeqRec.seq).lower()})
    # df = pd.DataFrame(ls)
    # print(df)

getSingleSequence("PA544053")

ID: PA544053.1
Name: PA544053
Description: WO 2022071435-A/1: SARS-CoV-2 PROTEIN-DERIVED PEPTIDES AND VACCINES INCLUDING THE SAME
Number of features: 1
/molecule_type=DNA
/topology=linear
/data_file_division=PAT
/date=29-JUL-2022
/accessions=['PA544053']
/sequence_version=1
/keywords=['WO 2022071435-A/1']
/source=Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
/organism=Severe acute respiratory syndrome coronavirus 2
/taxonomy=['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
/references=[Reference(title='SARS-CoV-2 PROTEIN-DERIVED PEPTIDES AND VACCINES INCLUDING THE SAME', ...)]
/comment=OS   Severe acute respiratory syndrome coronavirus 2 PN   WO
2022071435-A/1
PD   07-APR-2022
PF   29-SEP-2021 WO 2021JP035967
PR   30-SEP-2020 JP 2020-164630         ,30-APR-2021 JP
PR   JP2021/017159       ,
PR   25-AUG-2021 US 63/236927
PA   ONCOTHERAPY 